In [1]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import SparkSession
from pyspark.ml.classification import LinearSVC
from pyspark.ml.functions import vector_to_array
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from math import *
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import LDA

spark = SparkSession.builder.master("local[1]")\
          .appName("years")\
          .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [2]:
### Loading data

df_1990 = spark.read.option("header","true").option("delimiter", ",").csv("/Users/youssouf/Downloads/1990.csv")
df_1990.show(2)

df_1993 = spark.read.option("header","true").option("delimiter", ",").csv("/Users/youssouf/Downloads/1993.csv")
df_1993.show(2)

df_1994 = spark.read.option("header","true").option("delimiter", ",").csv("/Users/youssouf/Downloads/1994.csv")
df_1994.show(2)

df_1998 = spark.read.option("header","true").option("delimiter", ",").csv("/Users/youssouf/Downloads/1998.csv")
df_1998.show(2)

df_2007 = spark.read.option("header","true").option("delimiter", ",").csv("/Users/youssouf/Downloads/2007.csv")
df_2007.show(2)

df_2008 = spark.read.option("header","true").option("delimiter", ",").csv("/Users/youssouf/Downloads/2008.csv")
df_2008.show(2)


+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1990|    1|         3|        3|   1707|      1630|   1755|      1723|           US|       29

In [3]:
### Sample of data

df_sample_1990 = df_1990.sample(fraction = 0.006)
list_df = [df_1993 ,df_1994, df_1998, df_2007, df_2008]

for df in list_df :
    df_sample_tmp = df.sample(fraction = 0.006)
    df_final_tmp = df_sample_1990.union(df_sample_tmp)
        
df_final = df_final_tmp
df_final.show(2)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1990|    1|         6|        6|   1633|      1630|   1718|      1723|           US|       29

In [4]:
# Count Na values per column
df_final.select([count(when(isnull(c), c)).alias(c) for c in df_final.columns]).show()


+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|   0|    0|         0|        0|      0|         0|      0|         0|            0|        0

In [5]:
# Drop columns TailNum, 
df_final = df_final.drop("TailNum", "CancellationCode", "UniqueCarrier", "Origin", "Dest")


In [6]:
# Check na values
df_final.select([count(when(isnull(c), c)).alias(c) for c in df_final.columns]).show()


+----+-----+----------+---------+-------+----------+-------+----------+---------+-----------------+--------------+-------+--------+--------+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|FlightNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Distance|TaxiIn|TaxiOut|Cancelled|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+---------+-----------------+--------------+-------+--------+--------+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+
|   0|    0|         0|        0|      0|         0|      0|         0|        0|                0|             0|      0|       0|       0|       0|     0|      0|        0|       0|           0|           0|       0|            0|               

In [7]:
## Fill na values
df_final = df_final.na.fill(0)
df_final = df_final.fillna(0)
df_final = df_final.replace('null' , str(0))
df_final = df_final.replace('NA' , str(0))


In [8]:
df_final.show(2)

+----+-----+----------+---------+-------+----------+-------+----------+---------+-----------------+--------------+-------+--------+--------+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|FlightNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Distance|TaxiIn|TaxiOut|Cancelled|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+---------+-----------------+--------------+-------+--------+--------+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+
|1990|    1|         6|        6|   1633|      1630|   1718|      1723|       29|               45|            53|      0|      -5|       3|     182|     0|      0|        0|       0|           0|           0|       0|            0|               

In [9]:
liste_columns = ['Year','Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime','ArrTime',
 'CRSArrTime','FlightNum','ActualElapsedTime','CRSElapsedTime','AirTime',
 'ArrDelay','DepDelay','Distance','TaxiIn','TaxiOut','Cancelled','Diverted',
 'CarrierDelay','WeatherDelay','NASDelay', 'SecurityDelay','LateAircraftDelay']

# columns_to_non_cast = ["UniqueCarrier", "Origin", "Dest"]
# - set(columns_to_non_cast)
columns_to_cast = list(set(liste_columns))

for cl in columns_to_cast:
    df_final = df_final.withColumn(cl, col(cl).cast(DoubleType()))

In [10]:
df_final = df_final.withColumn("Delayed", when((col("ArrDelay") <= 0) & (col("DepDelay") <= 0), "N").otherwise("Y"))

In [11]:
df_final.show(2)

+------+-----+----------+---------+-------+----------+-------+----------+---------+-----------------+--------------+-------+--------+--------+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+-------+
|  Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|FlightNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Distance|TaxiIn|TaxiOut|Cancelled|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|Delayed|
+------+-----+----------+---------+-------+----------+-------+----------+---------+-----------------+--------------+-------+--------+--------+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+-------+
|1990.0|  1.0|       6.0|      6.0| 1633.0|    1630.0| 1718.0|    1723.0|     29.0|             45.0|          53.0|    0.0|    -5.0|     3.0|   182.0|   0.0|    0.0|      0.0|     0.0|         0.0|         0.0|     0

In [12]:
### Count Number of Delayed
df_final.groupBy("Delayed").count().show()

+-------+-----+
|Delayed|count|
+-------+-----+
|      Y|27821|
|      N|18210|
+-------+-----+



We are not in a case of unbalanced data

In [13]:
### Renaming N = 0 and Y = 1
df_final = df_final.withColumn("Delayed", when(col("Delayed") == "Y", 1).otherwise(0)) \
                    .withColumn("Delayed", col("Delayed").cast(DoubleType()))


df_final.show(2)

+------+-----+----------+---------+-------+----------+-------+----------+---------+-----------------+--------------+-------+--------+--------+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+-------+
|  Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|FlightNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Distance|TaxiIn|TaxiOut|Cancelled|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|Delayed|
+------+-----+----------+---------+-------+----------+-------+----------+---------+-----------------+--------------+-------+--------+--------+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+-------+
|1990.0|  1.0|       6.0|      6.0| 1633.0|    1630.0| 1718.0|    1723.0|     29.0|             45.0|          53.0|    0.0|    -5.0|     3.0|   182.0|   0.0|    0.0|      0.0|     0.0|         0.0|         0.0|     0

In [14]:
df_final.printSchema()

root
 |-- Year: double (nullable = true)
 |-- Month: double (nullable = true)
 |-- DayofMonth: double (nullable = true)
 |-- DayOfWeek: double (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- CRSDepTime: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- CRSArrTime: double (nullable = true)
 |-- FlightNum: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- Diverted: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)
 |

In [15]:
### Vectorisation
#unwanted = {'Delayed_New', 'Delayed', "UniqueCarrier", "Origin", "Dest", "TailNum", "CancellationCode"}
#inputcol = [e for e in liste_columns if e not in unwanted]

#string_indexer1 = StringIndexer(inputCol = "UniqueCarrier", outputCol = "UniqueCarrierIndex")

#string_indexer2 = StringIndexer(inputCol = "Origin", outputCol = "OriginIndex")

#string_indexer3 = StringIndexer(inputCol = "Dest", outputCol = "DestIndex")

#string_indexer4 = StringIndexer(inputCol = "TailNum", outputCol = "DestIndex")

#string_indexer5 = StringIndexer(inputCol = "CancellationCode", outputCol = "CancellationCodeIndex")

vector_assembler = VectorAssembler(inputCols=liste_columns , outputCol="features").setHandleInvalid("keep")


In [16]:
### Split train , test
train, test = df_final.randomSplit([0.75 , 0.25], seed=12345)

In [17]:
### Metrics of models
def metrics_model(method, label, train_data, test_data):
    skill = method

    train, test = train_data, test_data

    pipeline = Pipeline(stages=[vector_assembler, skill])

    model = pipeline.fit(train)

    predictions = model.transform(test)

    results = predictions.select(['prediction', label])

    predictionAndLabels = results.rdd

    metrics = MulticlassMetrics(predictionAndLabels)

    cm = metrics.confusionMatrix().toArray()

    accuracy = (cm[0][0] + cm[1][1]) / cm.sum()

    precision = (cm[0][0]) / (cm[0][0] + cm[1][0])

    recall = (cm[0][0]) / (cm[0][0] + cm[0][1])

    f1score = 2*((precision*recall)/(precision+recall))

    evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",
                                           labelCol=label)

    AUC = evaluator.evaluate(predictions)

    result = {"Accuracy": accuracy, "precision": precision, "recall": recall, "AUC": AUC, "F1score": f1score}

    dico_result = {"METRIQUES" : result, "MATRICE DE CONFUSION": cm }

    return dico_result

In [18]:
### Models

rf = RandomForestClassifier(labelCol="Delayed", featuresCol="features", numTrees=100, impurity='gini', maxBins=20)

dt = DecisionTreeClassifier(labelCol="Delayed", featuresCol="features", impurity='gini', maxDepth=2, maxBins=20)

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol='Delayed')


In [19]:
### Random Forest
metrics_model(rf, "Delayed", train, test)

{'METRIQUES': {'Accuracy': 1.0,
  'precision': 1.0,
  'recall': 1.0,
  'AUC': 0.999999873213097,
  'F1score': 1.0},
 'MATRICE DE CONFUSION': array([[4507.,    0.],
        [   0., 7000.]])}

In [20]:
### Decision Tree
metrics_model(dt, "Delayed", train, test)

{'METRIQUES': {'Accuracy': 1.0,
  'precision': 1.0,
  'recall': 1.0,
  'AUC': 1.0,
  'F1score': 1.0},
 'MATRICE DE CONFUSION': array([[4507.,    0.],
        [   0., 7000.]])}

In [21]:
### Logistic Regression
metrics_model(lr, "Delayed", train, test)

<ipython-input-17-7ec2887359af>:23: RuntimeWarning: invalid value encountered in double_scalars
  precision = (cm[0][0]) / (cm[0][0] + cm[1][0])


{'METRIQUES': {'Accuracy': 0.6083253671678109,
  'precision': nan,
  'recall': 0.0,
  'AUC': 0.5,
  'F1score': nan},
 'MATRICE DE CONFUSION': array([[   0., 4507.],
        [   0., 7000.]])}

In [7]:
from kafka import KafkaAdminClient

In [26]:
KafkaAdminClient(bootstrap_servers = 'localhost:9092').delete_topics(topics = ["test2"])

DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='test2', error_code=0)])

In [ ]:
kafka_servers = "localhost:9092"
client = KafkaClient(bootstrap_servers=kafka_servers)
# [...] getting broker_topics
admin_client = KafkaAdminClient(bootstrap_servers=kafka_servers)
if topic_name in broker_topics:
    deletion = admin_client.delete_topics([topic_name])
    try:
        future = client.cluster.request_update()
        client.poll(future=future)
    except KafkaError as e:
        print(e)
        pass
admin_client.create_topics(new_topics="test2", validate_only=False)

In [27]:
#KafkaAdminClient(bootstrap_servers = 'localhost:9092').create_topics(new_topics=["test2"], validate_only=False)